In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Importing all packages
import json
import nltk
import re
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def to_read_txt(filname):
  '''
  This function read the txt file line by line
  '''
  with open(filname, 'r') as file:
    lines = file.readlines()
    return lines

In [4]:
def to_read_json(jsonfilename):
  '''
  This function read the json file and store both Abbreviation and Synonyms in parameter_list
  '''
  parameter_list=[]
  with open(jsonfilename, 'r') as file:
    data = json.load(file)
  for eachdic in data:
      parameter_list.append(eachdic["Abbreviation"].lower())
      for eachsynonyms in eachdic["Synonyms"]:
        parameter_list.append(eachsynonyms.lower())
  return parameter_list

In [5]:
def has_numbers(lst):
  '''
  This function will detect if the line has numbers, if it has number it will give true or false
  '''
  for item in lst:
        if isinstance(item, (int, float)) or (isinstance(item, str) and (item.isdigit() or (item.replace('.', '', 1).isdigit() if '.' in item else False))):
            return True
  return False

In [6]:
def removeunwated_token(tokens):
  '''
  This function removes unwanted tokens in the line lik anything in (). Ex (3.5-6.0)
  '''
  first_index=0
  last_index=0
  for index,each in enumerate(tokens):
    if each =="(":
      first_index=index
    if each ==")":
      last_index=index
  if last_index!=0:
    tokens = tokens[:first_index] + tokens[last_index + 1:]
  return tokens

In [7]:
def detect_unit(word):
    '''
    This function will detect units, as units start consist of / or %
    Exception mg , L etc
    '''
    pattern = r'[/%]'
    if re.search(pattern, word):
        return True
    else:
        return False

In [8]:
def is_word_similar_to_parameters(word, parameter_list, threshold=0.7):
    '''
    This function uses cosine similary to check is parameter in txt is similar to parameters in json
    file, if  similary is greater than 0.75 it returns True
    '''
    vectorizer = CountVectorizer().fit(parameter_list + [word])
    parameter_vectors = vectorizer.transform(parameter_list)
    word_vector = vectorizer.transform([word])
    cosine_similarities = cosine_similarity(word_vector, parameter_vectors).flatten()
    if any(similarity >0.75 for similarity in cosine_similarities):
        return True
    return False

In [9]:
def find_similar_words(word_list, parameter_list, threshold=0.7):
    '''
    This function will return dic which consit of paramers and its similarity score
    '''
    vectorizer = CountVectorizer().fit(parameter_list + word_list)
    parameter_vectors = vectorizer.transform(parameter_list)
    word_list_vectors = vectorizer.transform(word_list)
    cosine_similarities = cosine_similarity(word_list_vectors, parameter_vectors)
    similar_words_mapping = {}
    for i, word in enumerate(word_list):
        word_similarity_scores = cosine_similarities[i]
        similar_words = [(parameter, similarity) for parameter, similarity in zip(parameter_list, word_similarity_scores) if similarity > threshold]
        if similar_words:
            similar_words_mapping[word] = similar_words
    return similar_words_mapping

In [10]:
def highest_similarity(similar_words_mapping):
    '''
    This function detect highest similar score from dic given from find_similar_words function
    '''
    highest_scores = {}
    for word, similarities in similar_words_mapping.items():
        max_similarity = max(similarities, key=lambda x: x[1])
        highest_scores[word] = max_similarity
    return highest_scores

In [11]:
def contains_special(word):
    '''
    The function contains_special filters data if it contains special symbols such as <, >, |, or -.
    '''
    if re.search(r'\b(?:\d+-\d+|[<>])\b', word):
        return True
    else:
        return False

In [12]:
def contains_and_or_between(text):
    '''
    This function will detect if lines has and , between words
    '''
    pattern = r'\b(?:and|between)\b'
    if re.match(pattern, text):
        return True
    else:
        return False

In [13]:
def contains_unwanted(word):
    if word in ["<",">","|"]:
        return True
    else:
        return False

In [14]:
def verify(each_list):
  for each_token in each_list:
      if contains_and_or_between(each_token) or contains_special(each_token) or contains_unwanted(each_token):
        return False
  return True

In [15]:
def makelist_from_lines(lines):
  '''
  This function make list of all lines from txt
  '''
  cleaned_list = [line for line in lines if line.strip()]
  return cleaned_list

In [16]:
def first_filter(cleaned_list):
  '''
  This function first_filter contains a data cleaning pipeline that utilizes regex and NLP techniques to remove unwanted lines from a given list of text data. The process involves several steps:

Tokenization: Each line of text is converted into tokens.

Lowercasing: All tokens are converted into lowercase for consistency.

Removal of Unwanted Tokens: Each list of tokens is passed to a function called remove_unwanted_tokens, which removes unwanted data such as (3.5-6.0) from the list of tokens.

Filtering Criteria: The filtered lines are determined based on three main conditions:

Condition 1: is_word_similar_to_parameters: Checks whether the line has parameters similar to those specified in a JSON file.
Condition 2: has_numbers: Determines if the line contains any numbers or floats.
Condition 3: detect_unit: Checks if there are any units present in the line, which are identified by characters like / and %.
The unwanted lines are removed based on these conditions, and the filtered lines are stored in a list called filtered_data.
'''
  filtered_data=[]
  for line in cleaned_list:
      tokens = nltk.word_tokenize(line)
      tokens_lower = [token.lower() for token in tokens]
      tokens_filtered=removeunwated_token(tokens)
      for eachtoken in tokens_lower :
        if is_word_similar_to_parameters(eachtoken, parameter_list) and has_numbers(line) and detect_unit(line) :
          filtered_data.append(tokens_filtered)
  return filtered_data

In [17]:
def second_filter(filtered_data):
  '''
  This function, second_filter, serves as the second layer of filtering in the cleaning process. Each line of data is passed into two separate functions, verify, which filter out unwanted data based on specific conditions.

Conditions for Filtering
Condition 1: Contains Special Symbols
The function contains_special filters data if it contains special symbols such as <, >, |, or -.

Condition 2: Contains "AND" or "BETWEEN"
The function contains_and_or_between filters data if it contains the words "and" and "between" within the line.

After passing through these conditions, the filtered data is stored in a list called new_filtered_data.
'''
  new_filtered_data=[]
  for each_list in filtered_data:
    if verify(each_list):
      new_filtered_data.append(each_list)
  return new_filtered_data


In [18]:
def extract_information(new_filtered_data):
  '''
  This function extract information like parameter, units and value from text using regex, NLP and coine similary method
  '''
  main_dic=[]
  for each_list in new_filtered_data:
      sub_dic={}
      similar_words_mapping = find_similar_words(each_list, parameter_list)
      highest_scores = highest_similarity(similar_words_mapping)
      max_limit=0
      max_value=0
      for i in highest_scores.values():
        if i[1]>max_limit:
          max_limit=i[1]
          max_value=i[0]
      sub_dic["parameter"]=max_value

      for each_token in each_list:
        if  detect_unit(each_token):
          index = each_list.index(each_token)
          try:
            if isinstance(float(each_list[index+1]), (int, float)):
              sub_dic["value"]=each_list[index+1]
          except:
            pass
          try:
            if isinstance(float(each_list[index-1]), (int, float)):
              sub_dic["value"]=each_list[index-1]
          except:
            pass
          sub_dic["unit"]=each_token
      main_dic.append(sub_dic)
  return main_dic


In [19]:
def remove_duplicate_parameter(main_dic):
  '''
  This function removes duplicate parameter and keeps unique parameter
  '''
  seen_parameters = set()
  Final_dic = []
  for item in main_dic:
      parameter_name = item.get('parameter')
      if parameter_name not in seen_parameters:
          Final_dic.append(item)
          seen_parameters.add(parameter_name)


  return Final_dic


In [20]:
filname="/content/drive/MyDrive/Entropy-Technologies/0ab9800e-bc9a-4388-aaa2-d4fc05e7d111.txt"
jsonfilename="/content/drive/MyDrive/Entropy-Technologies/X1.json"


lines=to_read_txt(filname)
parameter_list=to_read_json(jsonfilename)
cleaned_list=makelist_from_lines(lines)
filtered_data=first_filter(cleaned_list)
new_filtered_data=second_filter(filtered_data)
main_dic=extract_information(new_filtered_data)
Final_dic=remove_duplicate_parameter(main_dic)
print(Final_dic)
print(len(Final_dic))

[{'parameter': 'tsh', 'value': '0.73', 'unit': 'miIU/L'}, {'parameter': 'ft4', 'value': '15', 'unit': 'pmol/L'}, {'parameter': 'ft3', 'value': '4.8', 'unit': 'pmol/L'}, {'parameter': 'vitamin a', 'value': '108', 'unit': 'nmol/L'}, {'parameter': 'cortisol', 'value': '188', 'unit': 'nmol/L'}, {'parameter': 'iron', 'value': '40', 'unit': 'umol/L'}, {'parameter': "t'ferrin", 'value': '24', 'unit': 'umol/L'}, {'parameter': 't. sat.', 'value': '85', 'unit': '%'}, {'parameter': 'ferritin', 'value': '63', 'unit': 'ug/L'}, {'parameter': 'anti-thyroglobulin', 'value': '110', 'unit': 'TU/mL'}, {'parameter': 'cea', 'value': '4.2', 'unit': 'ug/L'}, {'parameter': 'ca 19-9', 'value': '10', 'unit': 'U/mL'}, {'parameter': 'ca125', 'value': '17', 'unit': 'U/mL'}, {'parameter': 'b12', 'value': '252', 'unit': 'pmol/L'}, {'parameter': 'folate', 'value': '23.1', 'unit': 'nmol/L'}, {'parameter': 'na+', 'value': '140', 'unit': 'mmol/L'}, {'parameter': 'cl-', 'value': '106', 'unit': 'mmol/L'}, {'parameter': 'h